<a href="https://colab.research.google.com/github/m10k1/gemma_cookbook/blob/main/Gemma_Common_usecase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemma 一般的なユースケース

このノートブックではGemmaが解決できる基本的なタスクについてのデモです。



In [1]:
import os
from google.colab import userdata

In [2]:
!pip install -q -U keras keras-nlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.1/705.1 kB 36.1 MB/s eta 0:00:00


## Exploring prompting capabilities

### Gemma

#### Gemmaについて

Gemmaは、Googleが提供する軽量で最先端のオープンモデルのファミリーで、Geminiモデルの作成に使用されたのと同じ研究と技術に基づいて構築されています。 Gemmaはテキスト対テキスト、デコーダのみの大規模言語モデルであり、オープンウェイト、事前学習済みバリアント、インストラクションチューニング済みバリアントがあり、英語で利用可能です。 Gemmaモデルは、質問応答、要約、推論を含む様々なテキスト生成タスクに適しています。 比較的小さなサイズであるため、ラップトップやデスクトップ、独自のクラウドインフラストラクチャなど、リソースが限られた環境でも導入が可能で、最先端のAIモデルへのアクセスを民主化し、すべての人のイノベーションを促進します。

#### プロンプトフォーマット


インストラクション・チューニング（IT）モデルは、学習時と推論時の両方において、すべてのインストラクション・チューニング例に余分な情報を注釈する特定のフォーマッタを使って学習される。 フォーマッターには2つの目的がある：

*　システム、ユーザー、アシスタントの役割など、会話における役割を示す。
* 特にマルチターンの会話において。

以下では、Gemma が使用するコントロールトークンとその使用例を示す。 コントロール・トークンは、私たちのトークナイザで予約されており、私たちのトークナイザに固有であることに注意してください。

* ユーザーターンを示すトークン：user
* モデルのターンを示すトークン： model
* 対話ターンの開始を示すトークン： \<start_of_turn>
* 対話ターンの終了を示すトークン： \<end_of_turn>




In [3]:
import keras
import keras_nlp
from IPython.display import display, Markdown, Latex

os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")


In [4]:

# Let's load Gemma using Keras
gemma_model_id = "gemma_1.1_instruct_2b_en"
gemma = keras_nlp.models.GemmaCausalLM.from_preset(gemma_model_id)

100%|██████████| 785/785 [00:00<00:00, 782kB/s]


100%|██████████| 4.67G/4.67G [00:33<00:00, 151MB/s]


100%|██████████| 591/591 [00:00<00:00, 1.33MB/s]


100%|██████████| 4.04M/4.04M [00:00<00:00, 103MB/s]


In [5]:
prompt = """user

Text:
Tech giant Google, originally a Stanford research project by Larry Page and Sergey Brin,
has become synonymous with searching the web. Their complex algorithms prioritize relevant
results, making Google the go-to platform for information seekers. Beyond search,
Google offers email (Gmail), document creation tools (Google Docs), mapping (Google Maps),
the Android operating system for smartphones, and the Chrome web browser.
A leader in AI, Google ventures into self-driving cars and quantum computing.
Their influence is so vast, "to google" has become a verb for searching online.

Could you summarize the text in a few words?
model"""

response = gemma.generate(prompt, max_length=512)
display(Markdown(response[len(prompt) :]))


Google is a technology giant known for its search engine, email, documents, maps, and more.

In [6]:
prompt = """user
Think and write your step-by-step reasoning before responding.

Explain why the sky is blue.
model"""

response = gemma.generate(prompt, max_length=1024)
display(Markdown(response[len(prompt) :]))



**Step 1: Light enters the atmosphere.**
- Sunlight, composed of various wavelengths of colors, enters the Earth's atmosphere from the sun.

**Step 2: Rayleigh Scattering.**
- When sunlight interacts with molecules in the atmosphere, certain wavelengths of light are scattered more efficiently than others.
- Blue light, with its shorter wavelengths, is scattered more extensively than other colors.

**Step 3: Blue light reaches our eyes.**
- As sunlight enters our eyes, the blue light is scattered away from our view, leaving behind the other colors that we perceive as colors.

**Step 4: Perception of blue sky.**
- The scattered blue light reaches our eyes and stimulates the cones in our retina, which interpret it as blue.
- This is why we experience the sky as blue during clear weather conditions.

In [7]:
prompt = """user
Classify the text into neutral, negative, or positive.
Generate only the class, nothing else.
Text: Text: I think the food was awesome.
model
Sentiment:"""

response = gemma.generate(prompt, max_length=64)
print("Sentiment: " + response[len(prompt) :])


Sentiment:  Positive


In [8]:
text = """Lost in a worn Berlin travel guide, Ania underlined Warsaw and London.
Two cities, two dreams: a bustling career in finance and
a cozy bookshop tucked away in a cobbled London street."""

prompt = f"""user
Your task is to find all the city names in the given text.
Your response is an array of the model names in the format [\"city_name\"].
If you don't find model names in the abstract or you are not sure, return [\"\"]

Text:
{text}
model"""

response = gemma.generate(prompt, max_length=512)
print("Extracted cities: " + response[len(prompt) :])

Extracted cities:  names: [""]

Answer:
["Berlin", "Warsaw", "London"]


In [9]:

import textwrap


def format_messages(messages):
    fmt_messages = [
        f"{author}\n{text}" for author, text in messages
    ]
    fmt_messages.append("model")
    return "\n".join(fmt_messages)


history = [
    (
        "user",
        "You're a helpful assistant. Answer user's question in a friendly and helpful way. "
        "Please introduce yourself first.",
    )
]

print("\n*** YOU'RE ENTERING CHAT MODE. TYPE 'EXIT' TO FINISH THE CHAT ***\n")
while True:
    prompt = format_messages(history)
    response = gemma.generate(prompt, max_length=512)
    response = response[len(prompt) :]

    history.append(("model", response))
    print(textwrap.fill(f"Gemma: {response} \n", 120))

    question = input("User: ")
    if question == "EXIT":
        break

    print()
    history.append(("user", question))


*** YOU'RE ENTERING CHAT MODE. TYPE 'EXIT' TO FINISH THE CHAT ***

Gemma:   Hi there! I'm your friendly AI assistant here to help you with any questions or tasks you may have. I'm here to
assist you with information, provide solutions, and make your experience more efficient. Let's get started!
User: Hello

Gemma:   Hello! I'm glad to be of assistance. I'm excited to learn more about how I can help you today. How can I assist
you?
User: EXIT


In [10]:
prompt = """user
Generate a Python function that multiplies two numbers
model"""

response = gemma.generate(prompt, max_length=1024)
display(Markdown(response[len(prompt) :]))


```python
def multiply(num1, num2):
  """
  This function multiplies two numbers.

  Args:
    num1: The first number.
    num2: The second number.

  Returns:
    The product of num1 and num2.
  """

  product = num1 * num2
  return product
```

**Usage:**

```python
# Multiply 5 and 10
product = multiply(5, 10)

# Print the product
print(product)  # Output: 50
```

**Explanation:**

* The function takes two parameters, `num1` and `num2`.
* Inside the function, we use the multiplication operator (`*`) to calculate the product of `num1` and `num2`.
* The product is stored in the `product` variable.
* The function returns the calculated product.

**Additional Notes:**

* This function only works for positive numbers. For negative numbers, a `ZeroDivisionError` will be raised.
* The function is a simple implementation of multiplication and does not handle any errors or special cases. For more complex multiplication scenarios, additional logic may be required.